In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

In [2]:
d = pd.read_csv('intersection congestion.csv')

In [3]:
d.columns

Index(['RowId', 'IntersectionId', 'Latitude', 'Longitude', 'EntryStreetName',
       'ExitStreetName', 'EntryHeading', 'ExitHeading', 'Hour', 'Weekend',
       'Month', 'Path', 'TotalTimeStopped_p20', 'TotalTimeStopped_p40',
       'TotalTimeStopped_p50', 'TotalTimeStopped_p60', 'TotalTimeStopped_p80',
       'TimeFromFirstStop_p20', 'TimeFromFirstStop_p40',
       'TimeFromFirstStop_p50', 'TimeFromFirstStop_p60',
       'TimeFromFirstStop_p80', 'DistanceToFirstStop_p20',
       'DistanceToFirstStop_p40', 'DistanceToFirstStop_p50',
       'DistanceToFirstStop_p60', 'DistanceToFirstStop_p80', 'City'],
      dtype='object')

In [4]:
d.iloc[random.sample(range(1,800000),5), 0:15]

,RowId,IntersectionId,Latitude,Longitude,EntryStreetName,ExitStreetName,EntryHeading,ExitHeading,Hour,Weekend,Month,Path,TotalTimeStopped_p20,TotalTimeStopped_p40,TotalTimeStopped_p50
699532,2619867,1148,39.92065,-75.17051,South Broad Street,South Broad Street,N,N,9,1,11,South Broad Street_N_South Broad Street_N,0,0,0
177811,2098146,133,42.35716,-71.06509,Beacon Street,Beacon Street,E,E,8,1,9,Beacon Street_E_Beacon Street_E,0,0,0
540257,2460592,366,39.94997,-75.17873,South 23rd Street,Locust Street,S,E,22,0,9,South 23rd Street_S_Locust Street_E,0,0,0
599609,2519944,646,39.97283,-75.14608,North 6th Street,North 6th Street,S,S,23,0,11,North 6th Street_S_North 6th Street_S,0,0,7
136422,2056757,459,33.75966,-84.38219,Piedmont Avenue Northeast,Piedmont Avenue Northeast,N,N,0,1,8,Piedmont Avenue Northeast_N_Piedmont Avenue No...,0,0,0


In [5]:
d.isna().sum()

RowId                         0
IntersectionId                0
Latitude                      0
Longitude                     0
EntryStreetName            8189
ExitStreetName             5534
EntryHeading                  0
ExitHeading                   0
Hour                          0
Weekend                       0
Month                         0
Path                          0
TotalTimeStopped_p20          0
TotalTimeStopped_p40          0
TotalTimeStopped_p50          0
TotalTimeStopped_p60          0
TotalTimeStopped_p80          0
TimeFromFirstStop_p20         0
TimeFromFirstStop_p40         0
TimeFromFirstStop_p50         0
TimeFromFirstStop_p60         0
TimeFromFirstStop_p80         0
DistanceToFirstStop_p20       0
DistanceToFirstStop_p40       0
DistanceToFirstStop_p50       0
DistanceToFirstStop_p60       0
DistanceToFirstStop_p80       0
City                          0
dtype: int64

In [6]:
d['NameMissing'] = 0
d.loc[(pd.isna(d['EntryStreetName'])==True) & (pd.isna(d['ExitStreetName'])==False), 'NameMissing'] = 1
d.loc[(pd.isna(d['EntryStreetName'])==False) & (pd.isna(d['ExitStreetName'])==True), 'NameMissing'] = 2
d.loc[(pd.isna(d['EntryStreetName'])==True) & (pd.isna(d['ExitStreetName'])==True), 'NameMissing'] = 3
d['NameMissing'].value_counts()

0    844259
1      7616
2      4961
3       573
Name: NameMissing, dtype: int64

In [7]:
# Intersection ID has duplicate in different cities. Create a unique ID.
d['InterIdNew'] = d['City']+'_'+d['IntersectionId'].astype(str)

In [8]:
d['TurnRoute'] = d['EntryHeading'] + '_' + d['ExitHeading']

In [9]:
# List of intersections which has missing value
InterMissingList = d.loc[d['NameMissing']>0, 'InterIdNew'].unique()
print(len(InterMissingList))
InterMissingList

188


array(['Atlanta_0', 'Atlanta_10', 'Atlanta_32', 'Atlanta_44',
       'Atlanta_53', 'Atlanta_83', 'Atlanta_133', 'Atlanta_146',
       'Atlanta_175', 'Atlanta_187', 'Atlanta_191', 'Atlanta_201',
       'Atlanta_203', 'Atlanta_211', 'Atlanta_214', 'Atlanta_231',
       'Atlanta_232', 'Atlanta_234', 'Atlanta_278', 'Atlanta_285',
       'Atlanta_299', 'Atlanta_300', 'Atlanta_305', 'Atlanta_307',
       'Atlanta_314', 'Atlanta_318', 'Atlanta_339', 'Atlanta_342',
       'Atlanta_344', 'Atlanta_372', 'Atlanta_376', 'Atlanta_378',
       'Atlanta_380', 'Atlanta_392', 'Atlanta_417', 'Atlanta_429',
       'Atlanta_453', 'Atlanta_479', 'Atlanta_500', 'Atlanta_512',
       'Boston_3', 'Boston_51', 'Boston_68', 'Boston_83', 'Boston_119',
       'Boston_121', 'Boston_128', 'Boston_145', 'Boston_150',
       'Boston_151', 'Boston_155', 'Boston_166', 'Boston_181',
       'Boston_215', 'Boston_239', 'Boston_333', 'Boston_336',
       'Boston_374', 'Boston_407', 'Boston_418', 'Boston_424',
       'Bosto

In [264]:
#d.loc[d['NameMissing']>0, 'InterIdNew'].value_counts()

In [10]:
def fill_enter(enter, turn, store):
    if pd.isna(enter) and turn in store:
        return store[turn][0]
    else:
        return enter

def fill_exit(exit, turn, store):
    if pd.isna(exit) and turn in store:
        return store[turn][1]
    else:
        return exit

In [11]:
for id in InterMissingList:
    store = {}
    data = d.loc[d['InterIdNew']==id, ['EntryStreetName', 'ExitStreetName', 'TurnRoute']]
    for i in range(data.shape[0]):
        row = data.iloc[i,:]
        if row.TurnRoute not in store and pd.isna(row.EntryStreetName)==False and pd.isna(row.ExitStreetName)==False:
            store[row.TurnRoute] = [row.EntryStreetName, row.ExitStreetName]
    d.loc[(d['InterIdNew']==id) & (pd.isna(d['EntryStreetName'])), 'EntryStreetName'] = d.loc[(d['InterIdNew']==id) & (pd.isna(d['EntryStreetName'])), :].apply(lambda x: fill_enter(x['EntryStreetName'], x['TurnRoute'], store), axis=1)
    d.loc[(d['InterIdNew']==id) & (pd.isna(d['ExitStreetName'])), 'ExitStreetName'] = d.loc[(d['InterIdNew']==id) & (pd.isna(d['ExitStreetName'])), :].apply(lambda x: fill_exit(x['ExitStreetName'], x['TurnRoute'], store), axis=1)

In [12]:
d.isna().sum()

RowId                         0
IntersectionId                0
Latitude                      0
Longitude                     0
EntryStreetName            7330
ExitStreetName             4465
EntryHeading                  0
ExitHeading                   0
Hour                          0
Weekend                       0
Month                         0
Path                          0
TotalTimeStopped_p20          0
TotalTimeStopped_p40          0
TotalTimeStopped_p50          0
TotalTimeStopped_p60          0
TotalTimeStopped_p80          0
TimeFromFirstStop_p20         0
TimeFromFirstStop_p40         0
TimeFromFirstStop_p50         0
TimeFromFirstStop_p60         0
TimeFromFirstStop_p80         0
DistanceToFirstStop_p20       0
DistanceToFirstStop_p40       0
DistanceToFirstStop_p50       0
DistanceToFirstStop_p60       0
DistanceToFirstStop_p80       0
City                          0
NameMissing                   0
InterIdNew                    0
TurnRoute                     0
dtype: i

In [13]:
d['NameMissing2'] = 0
d.loc[(pd.isna(d['EntryStreetName'])==True) & (pd.isna(d['ExitStreetName'])==False), 'NameMissing2'] = 1
d.loc[(pd.isna(d['EntryStreetName'])==False) & (pd.isna(d['ExitStreetName'])==True), 'NameMissing2'] = 2
d.loc[(pd.isna(d['EntryStreetName'])==True) & (pd.isna(d['ExitStreetName'])==True), 'NameMissing2'] = 3
d['NameMissing2'].value_counts()

# List of intersections which has missing value
InterMissingList2 = d.loc[d['NameMissing2']>0, 'InterIdNew'].unique()
print(len(InterMissingList2))
print(len(d['InterIdNew'].unique()))
InterMissingList2

159
4796


array(['Atlanta_0', 'Atlanta_10', 'Atlanta_32', 'Atlanta_44',
       'Atlanta_53', 'Atlanta_83', 'Atlanta_133', 'Atlanta_146',
       'Atlanta_175', 'Atlanta_187', 'Atlanta_191', 'Atlanta_201',
       'Atlanta_203', 'Atlanta_211', 'Atlanta_214', 'Atlanta_231',
       'Atlanta_232', 'Atlanta_234', 'Atlanta_285', 'Atlanta_299',
       'Atlanta_300', 'Atlanta_305', 'Atlanta_307', 'Atlanta_314',
       'Atlanta_318', 'Atlanta_339', 'Atlanta_342', 'Atlanta_344',
       'Atlanta_372', 'Atlanta_376', 'Atlanta_378', 'Atlanta_380',
       'Atlanta_392', 'Atlanta_417', 'Atlanta_429', 'Atlanta_453',
       'Atlanta_479', 'Atlanta_500', 'Atlanta_512', 'Boston_68',
       'Boston_83', 'Boston_119', 'Boston_128', 'Boston_145',
       'Boston_150', 'Boston_151', 'Boston_155', 'Boston_239',
       'Boston_333', 'Boston_336', 'Boston_374', 'Boston_407',
       'Boston_418', 'Boston_476', 'Boston_498', 'Boston_521',
       'Boston_553', 'Boston_563', 'Boston_565', 'Boston_692',
       'Boston_695', 'Bos

In [14]:
# driving route recognition
direction_degree = {'NW':315, 'SE':135, 'NE':45, 'SW':225, 'E':90, 'W':270, 'S':180, 'N':0}
d['EntryHeadingDegree'] = d['EntryHeading'].apply(lambda x: direction_degree[x])
d['ExitHeadingDegree'] = d['ExitHeading'].apply(lambda x: direction_degree[x])
d['TurnDegree'] = d['ExitHeadingDegree'] - d['EntryHeadingDegree']
d['TurnDegree'].unique()

array([   0,   90,  -90,   45,  225,  270, -135,  -45,  180,  315, -270,
       -180,  135, -225, -315], dtype=int64)

In [15]:
d_agg = d.groupby(['InterIdNew','ExitHeading'])['TurnDegree'].count()
InterStruc = d_agg.unstack()
InterStruc.fillna(0, inplace=True)
InterStruc[InterStruc>0] = 1
InterStruc['DirectionNum'] = InterStruc.apply(lambda x: x.sum(), axis=1)
InterStruc.reset_index(level=0, inplace=True)
print(InterStruc.sample(10))
print(InterStruc['DirectionNum'].value_counts())

ExitHeading         InterIdNew    E    N   NE   NW    S   SE   SW    W  \
1349                Boston_999  1.0  0.0  1.0  0.0  0.0  0.0  1.0  1.0   
4771          Philadelphia_961  0.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0   
2494              Chicago_2396  0.0  0.0  0.0  1.0  0.0  1.0  0.0  0.0   
4741          Philadelphia_918  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0   
2745              Chicago_2699  0.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0   
4643          Philadelphia_775  1.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0   
2097                Chicago_19  1.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0   
2257              Chicago_2093  0.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0   
3921         Philadelphia_1591  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0   
352                 Atlanta_59  0.0  0.0  1.0  1.0  0.0  1.0  1.0  0.0   

ExitHeading  DirectionNum  
1349                  4.0  
4771                  3.0  
2494                  2.0  
4741                  2.0  
2745                  2.0  
4643             

In [16]:
d_agg = d.groupby(['InterIdNew','TurnRoute'])['TurnDegree'].count()
RountStruc = d_agg.unstack()
RountStruc.fillna(0, inplace=True)
RountStruc[RountStruc>0] = 1
RountStruc['RountNum'] = RountStruc.apply(lambda x: x.sum(), axis=1) 
RountStruc.reset_index(level=0, inplace=True)
print(RountStruc.sample(10))
print(RountStruc['RountNum'].value_counts())

TurnRoute         InterIdNew  E_E  E_N  E_NE  E_NW  E_S  E_SE  E_SW  E_W  \
4473        Philadelphia_539  0.0  0.0   0.0   0.0  0.0   0.0   0.0  0.0   
4408        Philadelphia_448  1.0  0.0   0.0   0.0  1.0   0.0   0.0  0.0   
2833            Chicago_2816  0.0  0.0   0.0   0.0  0.0   0.0   0.0  0.0   
3759       Philadelphia_1377  1.0  1.0   0.0   0.0  1.0   0.0   0.0  0.0   
4115       Philadelphia_1851  0.0  0.0   0.0   0.0  0.0   0.0   0.0  0.0   
336               Atlanta_51  1.0  1.0   0.0   0.0  1.0   0.0   0.0  0.0   
4288        Philadelphia_292  1.0  0.0   1.0   0.0  0.0   0.0   0.0  0.0   
2046            Chicago_1839  0.0  0.0   0.0   0.0  0.0   0.0   0.0  0.0   
4242        Philadelphia_232  1.0  1.0   0.0   0.0  1.0   0.0   0.0  0.0   
3564       Philadelphia_1119  1.0  1.0   0.0   0.0  1.0   0.0   0.0  0.0   

TurnRoute  NE_E  ...  S_W  W_E  W_N  W_NE  W_NW  W_S  W_SE  W_SW  W_W  \
4473        0.0  ...  0.0  0.0  0.0   0.0   0.0  1.0   0.0   0.0  1.0   
4408        0.0  

In [17]:
inter = pd.merge(InterStruc[['InterIdNew', 'DirectionNum']], RountStruc[['InterIdNew', 'RountNum']], how='left', left_on='InterIdNew', right_on='InterIdNew')
print(inter.sample(10))
print(inter.groupby(['DirectionNum', 'RountNum'])['InterIdNew'].count())

             InterIdNew  DirectionNum  RountNum
1407       Chicago_1075           4.0       7.0
1692       Chicago_1402           2.0       2.0
3056        Chicago_503           4.0       4.0
3290        Chicago_778           2.0       2.0
294         Atlanta_458           4.0       7.0
3117        Chicago_573           2.0       2.0
2444       Chicago_2331           2.0       2.0
3843  Philadelphia_1484           2.0       3.0
4736   Philadelphia_913           3.0       6.0
2321       Chicago_2173           3.0       4.0
DirectionNum  RountNum
1.0           1.0          314
              2.0           58
              3.0            3
2.0           2.0         1116
              3.0          251
              4.0          408
              5.0           28
              6.0            8
              7.0            1
3.0           3.0          291
              4.0          280
              5.0          231
              6.0          281
              7.0          278
              8

In [18]:
d['TurnCategory'] = 'Straight'
d.loc[((d['TurnDegree']<=135) & (d['TurnDegree']>45)) | ((d['TurnDegree']<=135-360) & (d['TurnDegree']>45-360)), 'TurnCategory'] = 'Right'
d.loc[(d['TurnDegree']==180) | (d['TurnDegree']==180-360), 'TurnCategory'] = 'Uturn'
d.loc[((d['TurnDegree']<315) & (d['TurnDegree']>=225)) | ((d['TurnDegree']<315-360) & (d['TurnDegree']>=225-360)), 'TurnCategory'] = 'Left'

In [19]:
# time window aggregation
d['TimeSlot'] = 'Midnight(22:00-07:00)'
d.loc[d['Hour'].isin([7,8,9]), 'TimeSlot'] = 'MorningBusy(07:00-10:00)'
d.loc[d['Hour'].isin([16,17,18,19]), 'TimeSlot'] = 'EveningBusy(16:00-20:00)'
d.loc[d['Hour'].isin([20,21]), 'TimeSlot'] = 'NormalNight(20:00-22:00)'
d.loc[d['Hour'].isin([10,11,12,13,14,15]), 'TimeSlot'] = 'NormalDay(10:00-16:00)'

In [20]:
d1 = d.groupby(['InterIdNew', 'Weekend', 'TimeSlot', 'TurnCategory'])['TotalTimeStopped_p80'].median().unstack().reset_index()
d2 = d.groupby(['InterIdNew'])['Latitude', 'Longitude'].median()
data = pd.merge(left=d1, right=d2, how='inner', left_on='InterIdNew', right_on='InterIdNew')
data.sample(10)

,InterIdNew,Weekend,TimeSlot,Left,Right,Straight,Uturn,Latitude,Longitude
429,Atlanta_15,1,NormalDay(10:00-16:00),0.0,NaN,0.0,NaN,33.74690,-84.38384
26017,Philadelphia_1385,0,MorningBusy(07:00-10:00),62.5,22.0,52.0,NaN,39.91407,-75.15001
31121,Philadelphia_296,0,Midnight(22:00-07:00),51.0,11.0,0.0,NaN,40.04327,-75.05223
7198,Boston_302,1,NormalDay(10:00-16:00),NaN,0.0,32.0,NaN,42.29552,-71.07212
29210,Philadelphia_1823,0,EveningBusy(16:00-20:00),19.0,NaN,0.0,NaN,39.93890,-75.14816
19757,Chicago_2793,0,NormalDay(10:00-16:00),19.0,0.0,10.5,NaN,41.87030,-87.73513
25515,Philadelphia_1307,1,EveningBusy(16:00-20:00),NaN,NaN,22.0,NaN,39.95833,-75.17112
25494,Philadelphia_1304,0,NormalNight(20:00-22:00),9.0,34.0,26.0,NaN,39.94386,-75.16719
13475,Chicago_1262,0,MorningBusy(07:00-10:00),NaN,NaN,0.0,NaN,41.89477,-87.63712
34605,Philadelphia_786,0,Midnight(22:00-07:00),21.5,38.0,17.0,NaN,39.97468,-75.19934


In [21]:
# ignore Uturn
del data['Uturn']
data.sample(10)

,InterIdNew,Weekend,TimeSlot,Left,Right,Straight,Latitude,Longitude
23942,Philadelphia_1105,1,MorningBusy(07:00-10:00),NaN,NaN,15.0,39.97265,-75.17914
13322,Chicago_1224,0,EveningBusy(16:00-20:00),NaN,NaN,6.0,41.82378,-87.62160
33118,Philadelphia_570,0,NormalDay(10:00-16:00),81.0,39.0,69.0,39.89463,-75.23671
28033,Philadelphia_1656,0,NormalNight(20:00-22:00),NaN,NaN,14.0,39.97280,-75.21634
26155,Philadelphia_1403,0,NormalDay(10:00-16:00),50.0,25.0,25.0,39.94773,-75.19869
2697,Atlanta_435,1,MorningBusy(07:00-10:00),NaN,NaN,0.0,33.75543,-84.34920
4060,Boston_1057,0,Midnight(22:00-07:00),NaN,NaN,0.0,42.36717,-71.06502
2324,Atlanta_388,0,NormalNight(20:00-22:00),NaN,6.0,19.0,33.75147,-84.38538
15236,Chicago_1697,0,NormalNight(20:00-22:00),NaN,NaN,12.0,41.90337,-87.66649
30622,Philadelphia_23,0,MorningBusy(07:00-10:00),26.0,0.0,0.0,39.93600,-75.14686


In [28]:
data.to_csv('ProgressDataOutputArchive/data_agg.csv')